# Results

How to use:

* update model_path and model_names (at the END of the file)
* run all cells


In [4]:
import os

import numpy as np
import scipy
import scipy.stats
import torch
from torch.utils.data import DataLoader
from torchinfo import summary 
from codes.pt_data import ProteinLigand_3DDataset
from codes.raw_data import RawDataset

## Setup

In [5]:
# data
input_dir= f"{os.path.join(os.getenv('luh_ALL_CCFRWORK'))}/deep_learning/pafnucy/data/CoG_12" 
grid_spacing= 1.0 # distance between grid points in angstrom
max_dist= 12 # max distance from complex center
batch_size= 50

# normalisation (used during training)
partialcharge= {"m": -0.1401471346616745, "std": 0.4216829240322113}

## Data

In [6]:
def get_data(name, input_dir, max_dist, partialcharge, grid_spacing, batch_size):
    raw_data = RawDataset(input_dir, name, max_dist)
    raw_data.load_data()
    raw_data.set_normalization_params(partialcharge["m"], partialcharge["std"])
    raw_data.charge_normalization()
    print(raw_data)

    dataset = ProteinLigand_3DDataset(raw_data, grid_spacing=grid_spacing, rotations=None)
    no_of_samples = len(dataset)
    batch_size = min(no_of_samples, batch_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True,
                                 persistent_workers=True)    
    return dataloader, no_of_samples


train_dl, train_samples= get_data('training', input_dir, max_dist, partialcharge, grid_spacing, batch_size)
val_dl, val_samples= get_data('validation', input_dir, max_dist, partialcharge, grid_spacing, batch_size)
test_dl, test_samples= get_data('test', input_dir, max_dist, partialcharge, grid_spacing, batch_size)


training dataset with 13800 samples
	Partial charge normalization: m= -0.1401471346616745                     std= 0.4216829240322113

validation dataset with 3479 samples
	Partial charge normalization: m= -0.1401471346616745                     std= 0.4216829240322113

test dataset with 285 samples
	Partial charge normalization: m= -0.1401471346616745                     std= 0.4216829240322113



## Run model on dataset

In [7]:
def run(model, dataloader, no_of_samples, model_summary=False, sample_summary=False):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval() # model already on gpu
    if model_summary:
        summary(model, input_size=(10, 19, 25, 25, 25))
        
    affinities = np.empty(0, dtype=np.float32)
    predictions = []

    for (inputs, labels) in dataloader:
        inputs = inputs.to(device)

        with torch.set_grad_enabled(False):
            preds = model(inputs)

        affinities = np.append(affinities, labels.numpy())
        predictions = np.append(predictions, preds.cpu().detach().numpy())

    if sample_summary:
        print(f"Computed preds on {len(predictions)}/{len(affinities)} samples! (expected: {no_of_samples})")
    return affinities, predictions

## Analysis


In [14]:
def analyse(affinities, predictions, name):
    rmse = ((predictions - affinities) ** 2).mean() ** 0.5
    mae = (np.abs(predictions - affinities)).mean()
    corr = scipy.stats.pearsonr(predictions, affinities)
    # lr = LinearRegression()
    # lr.fit(predictions, affinities)
    # y_ = lr.predict(predictions)
    # sd = (((affinities - y_) ** 2).sum() / (len(affinities) - 1)) ** 0.5
    
    print(f"Analysis of {name} data: rmse= {rmse:.5f}\tmae= {mae:.5f}\tcorr= {corr}")

## Analysis of several models

* "rotations_2.2152_4.pth" : only save model once per epoch and at the end of the 24 rotations, if loss is better
* "rotations_saveinbetween_2.1320_e7_r0.pth" : check for better loss after each rotation, and save (here at rotation 0)


In [15]:
#model_path = f"{os.getenv('luh_ALL_CCFRSCRATCH')}/proli/models"
#model_names = ["rotations_2.2152_4.pth", "rotations_saveinbetween_2.1320_e7_r0.pth", "nil_rot_inbetween_2.1599_e4_r7.pth", "nil_rot_inb_noscheduler_2.1259_e3_r7.pth"]

model_path = f"{os.getenv('luh_ALL_CCFRSCRATCH')}/proli/nil_models"
model_names = ["nil_nomaxpool_mse2.3608_mae1.2189_e14.pth","nil_nomaxpool_aw_mse2.2175_mae1.1900_e62.pth","nil_nomaxpool_adw_sched_mse2.2676_mae1.1981_e15.pth"]

def perform_analysis(m):
    model_fullname = os.path.join(model_path, m)
    model = torch.load(model_fullname)
    
    affinities, predictions = {}, {}
    affinities["test"], predictions["test"] = run(model, test_dl, test_samples, model_summary=True, sample_summary=True)
    affinities["val"], predictions["val"] = run(model, val_dl, val_samples)
    affinities["train"], predictions["train"] = run(model, train_dl, train_samples)
    print(f"==== model {m} ====")
    for name in ["train", "val", "test"]:
        analyse(affinities[name], predictions[name], name)

In [16]:
for m in model_names:
    perform_analysis(m)

Computed preds on 285/285 samples! (expected: 285)
==== model nil_nomaxpool_mse2.3608_mae1.2189_e14.pth ====
Analysis of train data: rmse= 1.48658	mae= 1.15684	corr= (0.6066411381185601, 0.0)
Analysis of val data: rmse= 1.53649	mae= 1.21895	corr= (0.6187196600474285, 0.0)
Analysis of test data: rmse= 1.59345	mae= 1.29103	corr= (0.6830717857341011, 1.6556294187406492e-40)
Computed preds on 285/285 samples! (expected: 285)
==== model nil_nomaxpool_aw_mse2.2175_mae1.1900_e62.pth ====
Analysis of train data: rmse= 1.41828	mae= 1.10850	corr= (0.6411507198664433, 0.0)
Analysis of val data: rmse= 1.48914	mae= 1.19004	corr= (0.6425663415088806, 0.0)
Analysis of test data: rmse= 1.56267	mae= 1.24478	corr= (0.7329014569567869, 2.9500972151452334e-49)
Computed preds on 285/285 samples! (expected: 285)
==== model nil_nomaxpool_adw_sched_mse2.2676_mae1.1981_e15.pth ====
Analysis of train data: rmse= 1.48238	mae= 1.15879	corr= (0.6035270360672834, 0.0)
Analysis of val data: rmse= 1.50586	mae= 1.1980